### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/game_purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load, low_memory=False)
# print(list(purchase_data))
purchase_data



,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,101,Final Critic,4.19


## Player Count

* Display the total number of players


In [2]:
purchase_data_uniqueSN = purchase_data[['SN', 'Gender']].drop_duplicates()
total_players = len(purchase_data_uniqueSN['SN'])
print(f'The total players : {total_players}')

The total players : 576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
# find the total number of unique items and total purchases
unique_item = purchase_data['Item ID'].unique()
num_unique_item = len(unique_item)

total_num_purchase = purchase_data['Item ID'].count()

# average price and total price
ave_price = round(purchase_data['Price'].mean(), 2)
total_rev = round(purchase_data['Price'].sum(), 2)


# create dataframe for summary table
sum_tbl_data = {"Number of Unique Items" : num_unique_item,"Average Price" : ave_price,
"Number of Purchases" : total_num_purchase, "Total Revenue" : total_rev}
sum_tbl_df = pd.DataFrame(sum_tbl_data, index=[0])

# format the summary dataframe
sum_tbl_df['Average Price'] = sum_tbl_df['Average Price'].map("${:.2f}".format)
sum_tbl_df['Total Revenue'] = sum_tbl_df['Total Revenue'].map("${:,.2f}".format)
sum_tbl_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
unique_SN_gender_count = purchase_data_uniqueSN["Gender"].value_counts()
unique_SN_gender_df = pd.DataFrame(unique_SN_gender_count)
unique_SN_gender_df["Percentage of Players"] = round(unique_SN_gender_df["Gender"] / total_players * 100, 2)
unique_SN_gender_df["Percentage of Players"] = unique_SN_gender_df["Percentage of Players"].map('{:.2f}%'.format)
unique_SN_gender_df.rename(columns={"Gender" : "Total Count"})

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
# Group all duplicates by SN names and all Genders
uni_dir = purchase_data.groupby(['SN','Gender'])['Price'].unique()
uni_df = pd.DataFrame(uni_dir)
uni_df.reset_index(inplace=True)

#create two new columns in df to store data
uni_df["Sum per individual"] = ""
# uni_df["Aver per individual"] = ""

# Looping to find the total spending of each female
for each_purchase in range(len(uni_df["SN"])):
    uni_df["Sum per individual"][each_purchase] = uni_df["Price"][each_purchase].sum()
#     uni_df["Aver per individual"][each_purchase] = uni_df["Sum per individual"][each_purchase] / len(uni_df["Price"][each_purchase])

uni_df

,SN,Gender,Price,Sum per individual
0,Adairialis76,Male,[2.28],2.28
1,Adastirin33,Female,[4.48],4.48
2,Aeda94,Male,[4.91],4.91
3,Aela59,Male,[4.32],4.32
4,Aelaria33,Male,[1.79],1.79
...,...,...,...,...
571,Yathecal82,Female,"[1.87, 2.42, 1.93]",6.22
572,Yathedeu43,Male,"[3.75, 2.27]",6.02
573,Yoishirrala98,Female,[4.58],4.58
574,Zhisrisu83,Male,"[4.35, 3.54]",7.89


In [6]:
#===================== FOR FEMALE =======================

# Using loc function to filter out only female purchases
# SN names have duplicates
female_pur_data = purchase_data.loc[purchase_data["Gender"] == "Female", :]

# Total female purchase count and cost, including SN duplicates
total_female_purchase_count = len(female_pur_data["Gender"])
total_female_purchase_cost = female_pur_data["Price"].sum()


# Eliminate all SN duplicates for Female Category
uni_female_df = uni_df.loc[uni_df['Gender'] == "Female", :]
total_femaleSN_unique = len(uni_female_df)

# Calculate average purchase price
ave_pur_price_female = total_female_purchase_cost / total_female_purchase_count

# Calculate Avg Total Purchase per Persontotal_femaleSN_unique
avgtotal_purchase_per_female = total_female_purchase_cost / total_femaleSN_unique


In [7]:
#===================== FOR MALE =======================

# Using loc function to filter out only female purchases
# SN names have duplicates
male_pur_data = purchase_data.loc[purchase_data["Gender"] == "Male", :]

# Total female purchase count and cost, including SN duplicates
total_male_purchase_count = len(male_pur_data["Gender"])
total_male_purchase_cost = male_pur_data["Price"].sum()


# Eliminate all SN duplicates for Female Category
uni_male_df = uni_df.loc[uni_df['Gender'] == "Male", :]
total_maleSN_unique = len(uni_male_df)

# Calculate average purchase price
ave_pur_price_male = total_male_purchase_cost / total_male_purchase_count


# Calculate Avg Total Purchase per Person
avgtotal_purchase_per_male =  total_male_purchase_cost / total_maleSN_unique


In [8]:
#===================== FOR OTHER / NON-DISCLOSE =======================

# Using loc function to filter out only female purchases
# SN names have duplicates
nd_pur_data = purchase_data.loc[purchase_data["Gender"] == "Other / Non-Disclosed", :]

# Total female purchase count and cost, including SN duplicates
total_nd_purchase_count = len(nd_pur_data["Gender"])
total_nd_purchase_cost = nd_pur_data["Price"].sum()


# Eliminate all SN duplicates for Female Category
uni_nd_df = uni_df.loc[uni_df['Gender'] == "Other / Non-Disclosed", :]
total_ndSN_unique = len(uni_nd_df)

# Calculate average purchase price
ave_pur_price_nd = total_nd_purchase_cost / total_nd_purchase_count


# Calculate Avg Total Purchase per Person
avgtotal_purchase_per_nd = total_nd_purchase_cost / total_ndSN_unique


In [9]:
# Build summary data table
sum_table = pd.DataFrame(
    {"Gender" : ["Female", "Male", "Other / Non-Disclosed"],
     "Purchase Count" : [total_female_purchase_count, total_male_purchase_count, total_nd_purchase_count],
     "Average Purchase Price" : [ave_pur_price_female, ave_pur_price_male, ave_pur_price_nd],
     "Total Purchase Value" : [total_female_purchase_cost, total_male_purchase_cost, total_nd_purchase_cost],
     "Avg Total Purchase per Person" : [avgtotal_purchase_per_female, avgtotal_purchase_per_male, avgtotal_purchase_per_nd]       
    })

sum_table

# Use Map to re-format all price columns
sum_table["Total Purchase Value"] = sum_table["Total Purchase Value"].map("${:,.2f}".format)
sum_table["Average Purchase Price"] = sum_table["Average Purchase Price"].map("${:.2f}".format)
sum_table["Avg Total Purchase per Person"] = sum_table["Avg Total Purchase per Person"].map("${:.2f}".format)
sum_table.head()


,Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0,Female,113,$3.20,$361.94,$4.47
1,Male,652,$3.02,"$1,967.64",$4.07
2,Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [10]:
# create bin and labels for AGE
bin_label = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']
bin_age = [0, 9, 14, 19, 24, 29, 34, 39, purchase_data["Age"].max() + 10]

# Create table with unique SN names
age_dir = purchase_data.groupby(['SN','Age'])['Price'].unique()
age_df = pd.DataFrame(age_dir)
age_df.reset_index(inplace=True)



In [11]:
# Display Age Demographics Table
age_df["Age Ranges"] = pd.cut(age_df["Age"], bin_age, labels=bin_label)
age_table = age_df["Age Ranges"].value_counts()
age_table_df = pd.DataFrame(age_table)
age_table_df["Percentage of Players"] = round(age_table / age_table.sum()*100, 2).map("{:.2f}%".format)
age_table_df.columns = ["Total Count", "Percentage of Players"]
age_table_df
age_table_df.sort_index()

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [12]:
purchase_data["Age Ranges"] = pd.cut(purchase_data["Age"], bin_age, labels=bin_label)

# short_purchase_data = purchase_data[["SN", "Age", "Gender", "Price", "Age Ranges"]]

# purchase_data.to_csv("test.csv", sep=",")

purchase_data

groupgroup_age_df = purchase_data.groupby(['Age Ranges'])['Price'].unique()
groupgroup_age_df.head()
group_age_df = pd.DataFrame(groupgroup_age_df)

group_age_df

len(purchase_data["Age Ranges"])

780

In [15]:
len(group_age_df["Price"][0])

21

In [14]:
group_age_df["Purchase Count"] = ""

for each_age_r in range(len(group_age_df)):
    group_age_df["Purchase Count"][each_age_r] = len(list(group_age_df["Price"][each_age_r]))
group_age_df


,Price,Purchase Count
Age Ranges,,
<10,"[3.09, 2.38, 2.05, 4.6, 3.47, 3.65, 4.09, 1.61...",21
10-14,"[1.61, 4.09, 4.18, 4.35, 3.53, 3.75, 2.45, 3.5...",24
15-19,"[2.94, 4.4, 3.77, 3.03, 4.23, 3.3, 3.09, 1.63,...",83
20-24,"[3.53, 4.88, 3.27, 1.44, 3.61, 2.67, 1.1, 4.74...",135
25-29,"[4.23, 3.44, 3.79, 2.22, 1.56, 4.64, 2.96, 1.0...",73
30-34,"[4.64, 3.19, 3.15, 1.44, 2.84, 2.38, 4.23, 1.2...",56
35-39,"[2.18, 3.58, 4.86, 3.81, 4.48, 1.71, 3.44, 4.4...",34
40+,"[1.56, 3.4, 1.79, 1.33, 3.39, 1.61, 4.93, 4.23...",13


* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [16]:
p = [1,2,3,2,4,3]

In [19]:
unique(p)


NameError: name 'unique' is not defined

In [22]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Ranges
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24


array([1, 2, 3, 4])

In [34]:
dff_Group= purchase_data.groupby(['Age Ranges'])
dfCount = dff_Group.count()
dfSum = dff_Group.sum()

In [35]:
dfMerge = pd.merge(dfCount,dfSum, on='Age Ranges', how='outer')

In [36]:
dfMerge

,Purchase ID_x,SN,Age_x,Gender,Item ID_x,Item Name,Price_x,Purchase ID_y,Age_y,Item ID_y,Price_y
Age Ranges,,,,,,,,,,,
<10,23,23,23,23,23,23,23,10009,181,2437,77.13
10-14,28,28,28,28,28,28,28,10114,319,2887,82.78
15-19,136,136,136,136,136,136,136,54859,2284,12798,412.89
20-24,365,365,365,365,365,365,365,139573,7971,32411,1114.06
25-29,101,101,101,101,101,101,101,41072,2626,9146,293.00
30-34,73,73,73,73,73,73,73,26254,2291,6875,214.00
35-39,41,41,41,41,41,41,41,16596,1505,4137,147.67
40+,13,13,13,13,13,13,13,5333,540,1158,38.24


In [37]:
d={}

In [38]:
d['<10'] = ['Kenn']

In [39]:
myList = [23,28,136]

In [41]:
for ar in AgeRangeList:
    dfQuery = purchase_list['Item Name']==ar
    filteredPurchaseList = purchase_list[dfQuery]
    print(filteredPurchaseList['Price'].mean())

1
1
1


In [42]:
purchase_data.columns

Index(['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price',
       'Age Ranges'],
      dtype='object')